---

# Process the Performance Report

---

In [69]:
# package imports go here
import pandas as pd
import numpy as np
import fastparquet as fp
import os
import sys
import pickle
import matplotlib.pyplot as plt
import importlib
import config
import time

sys.path.insert(1, config.package_path)
import ml_analysis as mlanlys

In [70]:
start_time = time.time()

In [71]:
# Path to performance report
report_path = 'reports/'
performance_report              = report_path + 'performance_report.pkl'
performance_report_text_file    = report_path + 'performance_report.txt'

---

## 1. Read performance report

---

In [72]:
if os.path.exists(performance_report):
    # Load Performance Report
    with open(performance_report, 'rb') as file: perf_report = pickle.load(file)
    print(f"The file {performance_report} exists and the Performance Report was read successfully")
else:
    print(f"******************************************************")
    print(f"The file: '{performance_report}' DOES NOT EXIST")
    print(f"******************************************************")

The file reports/performance_report.pkl exists and the Performance Report was read successfully


---

## 2. Write the Performance Report

---

In [73]:

perf_datasets_in = perf_report.keys()

perf_datasets_in

dict_keys(['2.0 StandardScaler Dataset', '2.1 MinMaxScaler Dataset', '3 Binary Dataset', '4 RandomUnderSampler Dataset', '5 RandomOverSampler Dataset', '7 SMOTE Dataset', '8 SMOTEENN Dataset', '1 Base Dataset', '6 ClusterCentroids Dataset'])

#### 2.1 Modify the order of the reported datasets

---

In [74]:
# Use this cell to re-order the datasets for the report

perf_datasets = perf_datasets_in

In [75]:
# Remove bad datasets if needed
# del perf_report['Base Dataset']

#### 2.2 Print the report

---

In [76]:
def print_performance_report(perf_report):
    # Print header and then every performance report in the reports dictionary

    print(f"**********************************************************")
    print(f"Diabetes Predictions Performance Summary Report")
    print(f"**********************************************************")

    print(f"----------------------------------------")
    print(f"Report Summary")
    print(f"----------------------------------------")

    report_keys = list(perf_report.keys())
    report_keys.sort()
    
    for report in report_keys:
        print(f"Performance for: {report}")

    print(f"----------------------------------------")

    for report in report_keys:
        report_dict = perf_report[report]
        dataset_size = report_dict['dataset_size']
        report_df = report_dict['report']
        report_df.rename(columns={'new_column':'dataset'}, inplace=True)

        print(f"----------------------------------------")
        print(f"Performance for: {report}")
        print(f"Dataset Size:    {dataset_size[0]} Rows, {dataset_size[1]} Columns")
        print(f"----------------------------------------")
        print(f"{report_df.to_string(index=False)}")

In [77]:
# Generate the Performance Report and send prints to osc.stdout
with mlanlys.OutStreamCapture() as osc:
    print_performance_report(perf_report)

In [78]:
# osc.stdout contains the details of the performance report
# write the performance report to the performance_report_text_file
with open(performance_report_text_file, "w") as file:
    file.write(osc.stdout)

In [79]:
# Display the performance report here:
print(osc.stdout)

**********************************************************
Diabetes Predictions Performance Summary Report
**********************************************************
----------------------------------------
Report Summary
----------------------------------------
Performance for: 1 Base Dataset
Performance for: 2.0 StandardScaler Dataset
Performance for: 2.1 MinMaxScaler Dataset
Performance for: 3 Binary Dataset
Performance for: 4 RandomUnderSampler Dataset
Performance for: 5 RandomOverSampler Dataset
Performance for: 6 ClusterCentroids Dataset
Performance for: 7 SMOTE Dataset
Performance for: 8 SMOTEENN Dataset
----------------------------------------
----------------------------------------
Performance for: 1 Base Dataset
Dataset Size:    253680 Rows, 22 Columns
----------------------------------------
       dataset                      model slice  score  balanced_accuracy  roc_auc_score  Mean Squared Error  Accuracy  Precision  Recall  F1-score  Specificity  False Positive Rate  Ma

In [80]:
print(f"Completed: Execution Time %s seconds:" % round((time.time() - start_time),2) )

Completed: Execution Time 0.15 seconds:


## 3. Merge Performance Metrics into a single dataframe

In [81]:
# Concatenate all the Reports into one dataframe
df = pd.DataFrame()

report_keys = list(perf_report.keys())
report_keys.sort()

for report in report_keys:
    report_dict = perf_report[report]
    report_df = report_dict['report']
    df = pd.concat([df, report_df], ignore_index=True)

In [82]:
df

,dataset,model,slice,score,balanced_accuracy,roc_auc_score,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient
0,1 Base Dataset,KNeighborsClassifier,Train,0.8859,0.4969,0.9473,0.4034,0.9021,0.7339,0.4703,0.5732,0.9723,0.0277,NaN
1,1 Base Dataset,KNeighborsClassifier,Test,0.8205,0.3899,0.6198,0.6582,0.8370,0.3587,0.2303,0.2805,0.9341,0.0659,0.1995
2,1 Base Dataset,DecisionTreeClassifier,Train,0.9935,0.9687,0.9999,0.0227,0.9945,0.9988,0.9616,0.9798,0.9998,0.0002,NaN
3,1 Base Dataset,DecisionTreeClassifier,Test,0.7671,0.4068,0.5691,0.8074,0.7980,0.2897,0.3194,0.3038,0.8747,0.1253,0.1864
4,1 Base Dataset,RandomForestClassifier,Train,0.9935,0.9695,0.9995,0.0228,0.9944,0.9942,0.9659,0.9798,0.9991,0.0009,19.5698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,8 SMOTEENN Dataset,GradientBoostingClassifier,Test,0.8668,0.5705,0.8294,0.1332,0.8668,0.5562,0.1615,0.2503,0.9794,0.0206,0.2478
122,8 SMOTEENN Dataset,AdaBoostClassifier,Train,0.8645,0.5844,0.8269,0.1355,0.8645,0.5432,0.1956,0.2876,0.9732,0.0268,0.2678
123,8 SMOTEENN Dataset,AdaBoostClassifier,Test,0.8655,0.5822,0.8259,0.1345,0.8655,0.5320,0.1913,0.2814,0.9731,0.0269,0.2612
124,8 SMOTEENN Dataset,LogisticRegression,Train,0.8634,0.5684,0.8222,0.1366,0.8634,0.5400,0.1588,0.2454,0.9780,0.0220,0.2389


## 4. Filter and sort test results

In [83]:
def rank_results(df, maximize, limits):
    # Sort Dataframe by column to maximize

    # Create dataframe with only the test metrics
    test_df = df[df['slice'] == 'Test']

    limit_df = test_df.copy()

    # limit the rows based on the values in the limits dictionary above
    for limit in limits:
        # print(limit)
        limit_value = limits[limit]
        limit_df = limit_df[limit_df[limit]>limit_value]

    # sort by column_to_maximize fro highest to lowest
    sorted_df = limit_df.sort_values(by=maximize, ascending=False, inplace=False)

    return sorted_df


#### 4.1 Filter examples

In [84]:
maximize = 'Precision'
limits = {
    'balanced_accuracy': .5,
    'Accuracy': .8,
    'roc_auc_score': .6
}

maximize = 'Accuracy'
limits = {
    'balanced_accuracy': .5,
    'Precision': .5,
    'roc_auc_score': .6
}

maximize = 'F1-score'
limits = {
    'Accuracy': .8,
    'Precision': .1,
    'roc_auc_score': .1
}

#### Filter - Accuracy

In [85]:
df.columns

Index(['dataset', 'model', 'slice', 'score', 'balanced_accuracy',
       'roc_auc_score', 'Mean Squared Error', 'Accuracy', 'Precision',
       'Recall', 'F1-score', 'Specificity', 'False Positive Rate',
       'Matthews Correlation Coefficient'],
      dtype='object')

In [86]:
df_print_len = 20

In [87]:
# Sort Dataframe by column to maximize
maximize = 'Accuracy'
limits = {
    'balanced_accuracy': .5,
    'Precision': .2
}

sorted_df1 = rank_results(df, maximize, limits).head(df_print_len)

sorted_df1

,dataset,model,slice,score,balanced_accuracy,roc_auc_score,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient
51,3 Binary Dataset,GradientBoostingClassifier,Test,0.8670,0.5707,0.8301,0.1330,0.8670,0.5616,0.1617,0.2511,0.9798,0.0202,0.2498
121,8 SMOTEENN Dataset,GradientBoostingClassifier,Test,0.8668,0.5705,0.8294,0.1332,0.8668,0.5562,0.1615,0.2503,0.9794,0.0206,0.2478
53,3 Binary Dataset,AdaBoostClassifier,Test,0.8663,0.5837,0.8269,0.1337,0.8663,0.5428,0.1935,0.2853,0.9739,0.0261,0.2670
123,8 SMOTEENN Dataset,AdaBoostClassifier,Test,0.8655,0.5822,0.8259,0.1345,0.8655,0.5320,0.1913,0.2814,0.9731,0.0269,0.2612
55,3 Binary Dataset,LogisticRegression,Test,0.8649,0.5684,0.8232,0.1351,0.8649,0.5344,0.1589,0.2450,0.9779,0.0221,0.2378
125,8 SMOTEENN Dataset,LogisticRegression,Test,0.8641,0.5665,0.8217,0.1359,0.8641,0.5219,0.1559,0.2401,0.9772,0.0228,0.2309
117,8 SMOTEENN Dataset,RandomForestClassifier,Test,0.8602,0.5720,0.7991,0.1398,0.8602,0.4794,0.1742,0.2555,0.9698,0.0302,0.2276
47,3 Binary Dataset,RandomForestClassifier,Test,0.8600,0.5712,0.7990,0.1400,0.8600,0.4786,0.1725,0.2536,0.9699,0.0301,0.2260
49,3 Binary Dataset,ExtraTreesClassifier,Test,0.8526,0.5686,0.7778,0.1474,0.8526,0.4180,0.1764,0.2481,0.9607,0.0393,0.2020
119,8 SMOTEENN Dataset,ExtraTreesClassifier,Test,0.8524,0.5691,0.7768,0.1476,0.8524,0.4159,0.1782,0.2495,0.9600,0.0400,0.2022


In [88]:
maximize = 'Precision'
limits = {
    'balanced_accuracy': .5,
    'Accuracy': .8,
    'roc_auc_score': .6
}

sorted_df2 = rank_results(df, maximize, limits).head(df_print_len)

sorted_df2

,dataset,model,slice,score,balanced_accuracy,roc_auc_score,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient
51,3 Binary Dataset,GradientBoostingClassifier,Test,0.8670,0.5707,0.8301,0.1330,0.8670,0.5616,0.1617,0.2511,0.9798,0.0202,0.2498
121,8 SMOTEENN Dataset,GradientBoostingClassifier,Test,0.8668,0.5705,0.8294,0.1332,0.8668,0.5562,0.1615,0.2503,0.9794,0.0206,0.2478
53,3 Binary Dataset,AdaBoostClassifier,Test,0.8663,0.5837,0.8269,0.1337,0.8663,0.5428,0.1935,0.2853,0.9739,0.0261,0.2670
55,3 Binary Dataset,LogisticRegression,Test,0.8649,0.5684,0.8232,0.1351,0.8649,0.5344,0.1589,0.2450,0.9779,0.0221,0.2378
123,8 SMOTEENN Dataset,AdaBoostClassifier,Test,0.8655,0.5822,0.8259,0.1345,0.8655,0.5320,0.1913,0.2814,0.9731,0.0269,0.2612
125,8 SMOTEENN Dataset,LogisticRegression,Test,0.8641,0.5665,0.8217,0.1359,0.8641,0.5219,0.1559,0.2401,0.9772,0.0228,0.2309
117,8 SMOTEENN Dataset,RandomForestClassifier,Test,0.8602,0.5720,0.7991,0.1398,0.8602,0.4794,0.1742,0.2555,0.9698,0.0302,0.2276
47,3 Binary Dataset,RandomForestClassifier,Test,0.8600,0.5712,0.7990,0.1400,0.8600,0.4786,0.1725,0.2536,0.9699,0.0301,0.2260
103,7 SMOTE Dataset,RandomForestClassifier,Test,0.8518,0.6126,0.7980,0.1482,0.8518,0.4466,0.2815,0.3453,0.9437,0.0563,0.2756
107,7 SMOTE Dataset,GradientBoostingClassifier,Test,0.8394,0.6811,0.8213,0.1606,0.8394,0.4277,0.4619,0.4441,0.9003,0.0997,0.3508


In [89]:
# Sort Dataframe by column to maximize
maximize = 'F1-score'
limits = {
    'Accuracy': .5,
    'Precision': .1,
    'roc_auc_score': .1
}


sorted_df3 = rank_results(df, maximize, limits).head(df_print_len)
sorted_df3

,dataset,model,slice,score,balanced_accuracy,roc_auc_score,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient
109,7 SMOTE Dataset,AdaBoostClassifier,Test,0.7866,0.7241,0.8157,0.2134,0.7866,0.3520,0.6374,0.4535,0.8107,0.1893,0.3572
67,4 RandomUnderSampler Dataset,AdaBoostClassifier,Test,0.7348,0.7538,0.8312,0.2652,0.7348,0.3141,0.7800,0.4479,0.7276,0.2724,0.3688
83,5 RandomOverSampler Dataset,LogisticRegression,Test,0.7319,0.7439,0.8193,0.2681,0.7319,0.3155,0.7606,0.4460,0.7272,0.2728,0.3588
81,5 RandomOverSampler Dataset,AdaBoostClassifier,Test,0.7309,0.7443,0.8229,0.2691,0.7309,0.3150,0.7630,0.4459,0.7256,0.2744,0.3590
107,7 SMOTE Dataset,GradientBoostingClassifier,Test,0.8394,0.6811,0.8213,0.1606,0.8394,0.4277,0.4619,0.4441,0.9003,0.0997,0.3508
65,4 RandomUnderSampler Dataset,GradientBoostingClassifier,Test,0.7243,0.7548,0.8334,0.2757,0.7243,0.3073,0.7970,0.4436,0.7127,0.2873,0.3666
111,7 SMOTE Dataset,LogisticRegression,Test,0.7338,0.7463,0.8221,0.2662,0.7338,0.3125,0.7638,0.4435,0.7289,0.2711,0.3598
79,5 RandomOverSampler Dataset,GradientBoostingClassifier,Test,0.7193,0.7481,0.8270,0.2807,0.7193,0.3086,0.7882,0.4435,0.7079,0.2921,0.3602
69,4 RandomUnderSampler Dataset,LogisticRegression,Test,0.7328,0.7485,0.8264,0.2672,0.7328,0.3107,0.7702,0.4428,0.7269,0.2731,0.3613
61,4 RandomUnderSampler Dataset,RandomForestClassifier,Test,0.7100,0.7382,0.8123,0.2900,0.7100,0.2924,0.7771,0.4249,0.6993,0.3007,0.3409


In [91]:
# Find all rows that exist in all three filtered results.
df_columns = list(sorted_df2.columns)
merged_df = pd.merge(sorted_df1, sorted_df2, on=df_columns, how='inner')
merge_df2 = pd.merge(merged_df, sorted_df3, on=df_columns, how='inner')

merge_df2

,dataset,model,slice,score,balanced_accuracy,roc_auc_score,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient
0,7 SMOTE Dataset,RandomForestClassifier,Test,0.8518,0.6126,0.7980,0.1482,0.8518,0.4466,0.2815,0.3453,0.9437,0.0563,0.2756
1,7 SMOTE Dataset,ExtraTreesClassifier,Test,0.8409,0.6146,0.7805,0.1591,0.8409,0.4029,0.3013,0.3448,0.9280,0.0720,0.2599
2,5 RandomOverSampler Dataset,RandomForestClassifier,Test,0.8404,0.6161,0.7919,0.1596,0.8404,0.4147,0.3029,0.3501,0.9293,0.0707,0.2658
3,7 SMOTE Dataset,GradientBoostingClassifier,Test,0.8394,0.6811,0.8213,0.1606,0.8394,0.4277,0.4619,0.4441,0.9003,0.0997,0.3508
